# Introduction <a name="introduction"></a>

This notebook demonstrates creating, listing, managing external models, deploying, and managing lifecycle phases for  external models in IBM watsonx.governance.

It covers the setup of the model, various deployment methods, and effective tracking throughout its lifecycle. 

For detailed documentation on these features, please refer to the [IBM AI Factsheet documentation](https://s3.us.cloud-object-storage.appdomain.cloud/aifactsheets-client/index.html).


**Required Services:**
- `watsonx.governance`

**Required Packages:**
- **IBM Facts Client Python SDK (>=1.0.80)**


<a name="setupcloud"></a>

In [3]:
!pip install ibm-aigov-facts-client --quiet
!pip install -U python-dotenv --quiet

In [5]:
import os
import warnings
import wget
import shutil
import json
import pandas as pd
import matplotlib.pyplot as plt
from dotenv import load_dotenv
from ibm_watsonx_ai import APIClient
from ibm_aigov_facts_client import AIGovFactsClient,CloudPakforDataConfig,DeploymentDetails,TrainingDataReference,ExternalModelSchemas,ModelDetails
from IPython.display import display, Markdown

warnings.filterwarnings("ignore")
shutil.rmtree('./mlruns', ignore_errors=True)
load_dotenv()

False

- This sample will use IBM Cloud by default. If you prefer to IBM watsonx.goverance software, set `use_software=True`

- Flag `run_cleanup_at_end` offers option to delete created assets at the end of the notebook.The notebook will show URL to UI for model and model use case at certain cells. By dafault we set it to `run_cleanup_at_end=False` so you can access UI and see the changes. If you decide to cleanup assets at the end, set `run_cleanup_at_end=True` and remember cells showing links to UI will `NOT` work in that case.

In [6]:
use_software=False
run_cleanup_at_end=True

- `Experiment` can be customized.

In [7]:
experiment_name="External Model"

---
## Authentication Setup<a name="setup"></a>

### IBM Cloud  <a name="IBM-Cloud"></a>

Your Cloud API key can be generated by going to the Users section of the Cloud console. From that page, go to **Manage->Access(IAM)->API keys-> Create**. Give your key a name and click Create, then copy the created key and use as API_KEY.

NOTE: You can also get OpenScale API_KEY using IBM CLOUD CLI.

How to install IBM Cloud (bluemix) console: instruction

How to get api key using console:

```
bx login --sso
bx iam api-key-create 'my_key'
```
- Get relevant space id from UI `(Deployments -> Spaces-> open space -> Manage -Space GUID)`

In [8]:
if not use_software:
    API_KEY=os.getenv("CLOUD_API_KEY", "<if you can't use .env you can provide your value here>")


### IBM watsonx.goverance software<a name="Watsonx.Gov-Platform"></a>

- Service url is the watsonx.goverance software platform host URL. For skytap environment, it would be the internal nginx URL.
- You can either use user `password` or platform `apikey` to authenticate

In [9]:
if use_software:
    creds=CloudPakforDataConfig(service_url=os.getenv("CPD_SERVICE_URL", "<if you can't use .env you can provide your value here>"),
        username=os.getenv("CPD_USERNAME", "<if you can't use .env you can provide your value here>"),
        password=os.getenv("CPD_PASSWORD", "<if you can't use .env you can provide your value here>"))



## Client Initialization

- If running this notebook multiple times with same experiment name or anytime face error saying `Experiment with same name already exists`, use `set_as_current_experiment=True` when initiating client




In [ ]:
if use_software:
   facts_client = AIGovFactsClient(cloud_pak_for_data_configs=creds,experiment_name=experiment_name,set_as_current_experiment=True,external_model=True)
else: 
   facts_client = AIGovFactsClient(api_key=API_KEY,experiment_name=experiment_name,set_as_current_experiment=True,external_model=True)

In [9]:
facts_client.version

'1.0.79'

---
## Inventory Management

This section focuses on the creation and management of inventory on the Watsonx.Governance platform.
The inventory is a view where you can define an AI use case to request a new model, and then track the model and related assets through its lifecycle


For more information, refer to the [Inventory Managment documentation](https://s3.us.cloud-object-storage.appdomain.cloud/aifactsheets-client/doc_files/asset_model/Model%20Usecase/Inventory.html).

<h3 style="color:gold;">Create Inventory</h3>

This method is used to create an inventory item with a specified name and description.

If you are utilizing IBM Cloud, ensure that you provide the **`cloud_object_storage_name`** parameter to correctly associate the inventory item with a Cloud Object Storage (COS) instance.


#### -> Retrieves a list of cloud object storage instances.

In [ ]:
if not use_software:
    cloud_object_storage_details=facts_client.utilities.get_cloud_object_storage_instances()
    print(cloud_object_storage_details)


In [ ]:
if not use_software:
    def get_storage_name_by_name(cloud_details, name):
        for detail in cloud_details:
            if detail['Name'] == name:
                return detail['Name']  # Return only the name
        return None  # Return None if the name is not found
  
    
    # select the desired name from the above list 
    user_selected_name = 'CloudObjectStorage'
    cloud_object_storage_name = get_storage_name_by_name(cloud_object_storage_details, user_selected_name)

    if cloud_object_storage_name:
        print(cloud_object_storage_name)
    else:
        print(f"No cloud_object_storage found with the name '{user_selected_name}'")


In [ ]:
if not use_software:
    inventory=facts_client.assets.create_inventory(name="New Inventory",description="testing",cloud_object_storage_name=cloud_object_storage_name)
else:
    inventory=facts_client.assets.create_inventory(name="New Inventory",description="testing")

In [ ]:
inventory.get_info()

---

<h2>Inventory Collaborators</h2>

Inventories are inherently collaborative, allowing multiple users with distinct roles to participate in the inventory.

This section provides a detailed overview of the methods available for managing collaborators within each inventory item, including:

- Viewing the current list of collaborators
- Assigning new collaborators
- Defining roles for each collaborator
- Removing existing collaborators




<h3 style="color:gold;">Retrieve a list of collaborators for the inventory. </h3>

In [ ]:
# Get the list of collaborators
inventory.list_collaborators()



<h3 style="color:gold;">Assigning New Collaborators</h3>

To add new collaborators, you need either the `user_id` or the `access_group_id`, depending on the platform you're using.

#### In the Watsonx.Gov Platform:
You need either the `user_id` or the `access_group_id`. To find these:

1. **Find the `user_id`:**
   - Go to the Watsonx.Gov Platform.
   - Navigate to **Access Control**.
   - Select **Users**.
   - Locate the `User-ID` for the desired user.

2. **Find the `access_group_id`:**
   - Go to the Watsonx.Gov Platform.
   - Navigate to **Access Control**.
   - Select **Users Group**.
   - Find the desired group.
   - The `access_group_id` is located at the end of the URL, e.g., `usermgmt-ui/groups/10001`, where `10001` is the group ID.

#### In the  IBM Cloud:
You need either the `user_iam_id` or the `access_group_id`. To find these:

1. **Find the `user_iam_id`:**
   - Go to [cloud.ibm.com](https://cloud.ibm.com).
   - Navigate to **IAM**.
   - Select **Users**.
   - Click on the three dots next to the user and choose **Manage User**. This will allow you to retrieve the necessary `user_iam_id` to add collaborators.

2. **Find the `access_group_id`:**
   - Go to [cloud.ibm.com](https://cloud.ibm.com).
   - Navigate to **IAM**.
   - Select **Access Groups**.
   - Click on the three dots next to the group and choose **Manage Access**. 
   - This will allow you to retrieve the necessary `access_group_id` from the details to add collaborators.




In [ ]:
# Determine user_id and access_group_id based on use_software flag
if use_software:
    user_id = "1000000**"
    access_group_id = "1000*"
else:
    user_id = "IBMid-69*******"
    access_group_id = "AccessGroupId-**********"


In [ ]:
if user_id:
    inventory.add_collaborator(user_id=user_id, role="editor")
elif access_group_id:
    inventory.add_collaborator(access_group_id=access_group_id, role="editor")
else:
    raise ValueError("Either user_id or access_group_id must be provided")

---
## Create External Model Properties <a name="createprop"></a>

- To demonstrate custom facts, we need to create a placeholder model on which we can show the APIs.
- The model itself does not matter for custom fact demonstration, so we will just create a sample one quickly.
- In a real end-to-end scenario, the model would be created based on real SageMaker/Azure/Custom information and use real names and IDs from these environments.

For more details, see the [product documentation](https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/factsheet-external.html) and the [Python documentation](https://s3.us.cloud-object-storage.appdomain.cloud/aifactsheets-client/doc_files/asset_model/Model/External.html).



### Essential Information Required for External Model Integration


- Input and Output schema details
- Training data reference information
- Model deployment details from the platform where the model is stored


### <span style="color: #FFD700;">Input and Output Schema Details</span>


In [10]:
!rm external_model_facts_payload.json
wget.download("https://raw.githubusercontent.com/IBM/ai-governance-factsheet-samples/main/Assets/data/external_model_facts_payload.json")

'external_model_facts_payload.json'

In [11]:
with open("external_model_facts_payload.json", 'r') as f:
    payload = json.load(f)

In [12]:
external_schemas=ExternalModelSchemas(input=payload["schemas"]["input"],output=payload["schemas"]["output"])

### <span style="color: #FFD700;">Training Data Reference Details</span> <a name="htd"></a>

The **Training Data Reference** is crucial for specifying the details and configuration required to access and utilize training data. This includes defining parameters such as the `schema, unique identifiers, type of data, and connectivity details`. Properly configuring these aspects ensures accurate and effective integration of training data, which is essential for building and deploying external models.


In [13]:
# Derive training data reference schema from model training input schema

training_data_schema={}

input_schema= payload["schemas"]["input"][0]
training_data_schema["schema"]=input_schema

In [14]:
TRAINING_DATA_ASSSET_NAME = "german_credit_data_biased_training.csv"
TYPE = "url"

External_data_url = "https://github.com/IBM/watson-openscale-samples/blob/main/IBM%20Cloud/WML/assets/data/credit_risk/german_credit_data_biased_training.csv"

- The **TrainingDataReference()** function is employed for creating the train_data_ref, which includes parameters like training schema, ID, type, and connectivity details.
- This formal approach ensures the accurate configuration and connection of training data for seamless external model creation.

In [15]:
tdataref = TrainingDataReference(schema=training_data_schema, id=TRAINING_DATA_ASSSET_NAME, type=TYPE, connection={"url":External_data_url})

### <span style="color: #FFD700;">Model Deployment Details and Model Details</span>

- **Model Deployment Details**: Includes parameters such as the identifier, name, deployment type, scoring endpoint, and description.

- **Model Details**: Covers attributes like the model type, input type, algorithm, label column, label type, and prediction type.


In [16]:
endpoint_name="sagemaker-scikit-learn-2023-08-11-09-00-35-455"
endpoint_url = "https://runtime.sagemaker.us-east-2.amazonaws.com/endpoints/sagemaker-scikit-learn-2023-08-11-09-00-35-455/invocations"
deployment_name = "Loan Application Model Deployment"

#### Model Deployment Details

In [17]:
deployment=DeploymentDetails(identifier=endpoint_name,name=deployment_name,deployment_type="online",scoring_endpoint=endpoint_url,description="Model deployed in AWS SageMaker to predict the loan application approval")



#### Model Details

In [18]:
model_details = ModelDetails(model_type="scikit-learn_1.1",input_type="Structured", algorithm="GradientBoost Algorithm",label_column="Risk", label_type="String",prediction_type="Binary")

## Save External Model to Inventory <a name="save"></a>

- This function facilitates the storage of external models within the inventory. It requires essential parameters including `endpoint_name`, `deployment`, and `model_details`.


- If the `Inventory_id` is not specified, the function will use the default Inventory ID from the platform's available assets. Nevertheless, all other specified parameters must be provided.

##### ⚠️ Warning: Use of `catalog_id`

- The `catalog_id` parameter can specify either an inventory ID or a catalog ID. Inventories, optimized for watsonx.governance, are recommended.
- Catalogs are still supported but will be deprecated over time. As a best practice, use inventories for storing use cases and external models.

For more details, see the [product documentation](https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/factsheet-external.html) and the [Python documentation](https://s3.us.cloud-object-storage.appdomain.cloud/aifactsheets-client/doc_files/asset_model/Model/External.html).


In [19]:
model_name = "Loan Application Model Test"
inventory_id=inventory.get_id()

In [20]:
external_model=facts_client.external_model_facts.save_external_model_asset(model_name,model_name,model_details=model_details,training_data_reference=tdataref,schemas=external_schemas,deployment_details=deployment,description="Model developed in AWS SageMaker to predict the loan application approval",catalog_id=inventory_id)

2024/08/09 21:08:00 INFO : External model asset saved successfully under asset_id 482d8c74-2655-4b17-aadb-eb45e1f56467 and catalog f8925e43-e661-4df7-8482-b6245433942c
2024/08/09 21:08:02 INFO : Current model information: {'asset_id': '482d8c74-2655-4b17-aadb-eb45e1f56467', 'container_type': 'catalog', 'container_id': 'f8925e43-e661-4df7-8482-b6245433942c', 'facts_type': 'modelfacts_user'}


In [22]:
external_model.get_info(True)

{'name': 'Loan Application Model Test',
 'description': 'Model developed in AWS SageMaker to predict the loan application approval',
 'asset_type': 'model_stub',
 'url': 'https://dataplatform.cloud.ibm.com/data/catalogs/f8925e43-e661-4df7-8482-b6245433942c/asset/482d8c74-2655-4b17-aadb-eb45e1f56467/asset-preview?context=cpdaas',
 'asset_id': '482d8c74-2655-4b17-aadb-eb45e1f56467',
 'container_type': 'catalog',
 'container_id': 'f8925e43-e661-4df7-8482-b6245433942c',
 'facts_type': 'modelfacts_user'}

---
## Fetch All External Models

<span style="color:lightblue">If an external model has already been created in the External Model section and you need to retrieve it, use this section.</span>


This section is used to fetch and display all externals models available within the `watsonx.governance`

The function retrieves external models from a specified inventory. By default, it returns up to `100 models`. To fetch a larger set of models, you can use additional parameters to refine your request.

#### Default Behavior

| **Scenario**                      | **Result**                                                                                             |
|-----------------------------------|--------------------------------------------------------------------------------------------------------|
| No Parameters                     | Returns up to 100 models from the specified or default inventory.                                      |
| With `start_index` and `max_results` | Returns models within the specified range. For example, `start_index=51` and `max_results=50` returns models 51 through 100. |


For more details, see the [List External Models documentation](https://s3.us.cloud-object-storage.appdomain.cloud/aifactsheets-client/doc_files/asset_model/Model/External.html#:~:text=list_external_models(inventory_id%3A%20str%20%3D%20None%2C%20start_index%3A%20int%20%3D%20None%2C%20max_results%3A%20int%20%3D%20None)).

#### <span style="color: gold;">Scenario 1: Searching External Models</span>

This scenario covers the approach for retrieving and processing up to 100 models, which can be managed in a single request without the need for additional paramters.


In [ ]:
ext_models=facts_client.external_model_facts.list_external_models()
print(ext_models)


In [ ]:
desired_model_name = "Desired External Model"

inventory_id = None
asset_id = None

for model in ext_models:
    if model.get('name') == desired_model_name:
        # Extract the inventory_id and asset_id
        inventory_id = model.get('inventory_id')
        asset_id = model.get('asset_id')
        break  # Exit the loop once the model is found

if inventory_id and asset_id:
    print(f"Model found: {model}")
else:
    print(f"Model with name '{desired_model_name}' not found.")


In [ ]:
if inventory_id and asset_id:
    container_type = 'catalog' 
    container_id = inventory_id 

    external_model =facts_client.assets.get_model(model_id=asset_id, container_type=container_type, container_id=container_id)
    print(f"Model details: {external_model}")
else:
    print("Model not found. Ensure that `inventory_id` and `asset_id` are correctly set.")


#### <span style="color: gold;"> Scenario 2: Searching External Models with Inventory ID</span>

In [ ]:
inventory_id = ext_models[0]['inventory_id']
print(inventory_id)

In [ ]:
ext_models_with_inventory = facts_client.external_model_facts.list_external_models(inventory_id=inventory_id)
print(ext_models_with_inventory)

In [ ]:
desired_model_name = "Desired External Model"

inventory_id = None
asset_id = None

for model in ext_models_with_inventory:
    if model.get('name') == desired_model_name:
        # Extract the inventory_id and asset_id
        inventory_id = model.get('inventory_id')
        asset_id = model.get('asset_id')
        break  # Exit the loop once the model is found

if inventory_id and asset_id:
    print(f"Model found: {model}")
else:
    print(f"Model with name '{desired_model_name}' not found.")


In [ ]:
if inventory_id and asset_id:
    container_type = 'catalog' 
    container_id = inventory_id 

    external_model =facts_client.assets.get_model(model_id=asset_id, container_type=container_type, container_id=container_id)
    print(f"Model details: {external_model}")
else:
    print("Model not found. Ensure that `inventory_id` and `asset_id` are correctly set.")


#### <span style="color: gold;"> Scenario 3:  Searching External Models with  Using `start_index` and `max_results`</span>

This scenario demonstrates how to manage and retrieve  External models by using the `start_index` and `max_results` parameters. This approach allows for efficient handling and processing of extensive model lists, ensuring that all relevant models are accessed without overloading the system.








In [ ]:
inventory_id = ext_models[0]['inventory_id']
print(inventory_id)

In [ ]:
ext_models_with_start_index=facts_client.external_model_facts.list_external_models(inventory_id=inventory_id,start_index=20,max_results=5)
print(ext_models_with_start_index)

In [ ]:
desired_model_name = "Desired External Model"

inventory_id = None
asset_id = None

for model in ext_models_with_start_index:
    if model.get('name') == desired_model_name:
        # Extract the inventory_id and asset_id
        inventory_id = model.get('inventory_id')
        asset_id = model.get('asset_id')
        break  # Exit the loop once the model is found

if inventory_id and asset_id:
    print(f"Model found: {model}")
else:
    print(f"Model with name '{desired_model_name}' not found.")


In [ ]:
if inventory_id and asset_id:
    container_type = 'catalog' 
    container_id = inventory_id 

    external_model =facts_client.assets.get_model(model_id=asset_id, container_type=container_type, container_id=container_id)
    print(f"Model details: {external_model}")
else:
    print("Model not found. Ensure that `inventory_id` and `asset_id` are correctly set.")


#### <span style="color: gold;">Scenario 4: Searching Large External Model Lists with `start_index` and `max_results`</span>

This scenario demonstrates how to manage and retrieve a large number of external models by using the `start_index` and `max_results` parameters. This approach allows for efficient handling and processing of extensive model lists, ensuring that all relevant models are accessed without overloading the system.


In [ ]:
ext_models = []

def fetch_external_models(inventory_id, start_index, max_results):
    try:
        response = facts_client.external_model_facts.list_external_models(
            inventory_id=inventory_id,
            start_index=start_index,
            max_results=max_results
        )
        return response
    except Exception as e:
        print(f"An error occurred while fetching models: {e}")
        return []

def paginate_external_models(inventory_id, start_index, max_results):
    models = fetch_external_models(inventory_id, start_index, max_results)
    
    if not isinstance(models, list):
        print("Unexpected format: models should be a list")
        return start_index
    
    if not models:
        print("No more models to fetch.")
        return None
    
    global ext_models
    ext_models = models

    new_start_index = start_index + max_results
    return new_start_index

def search_model_by_name(name):
    global ext_models
    for model in ext_models:
        if name.lower() in model.get('name', '').lower():
            return model
    return None

def final_external_model_with_start_index_max_results(inventory_id, max_results, search_name):
    start_index = 0

    while True:
        # Fetch models and check if pagination should continue
        start_index = paginate_external_models(inventory_id, start_index, max_results)
        if start_index is None:
            break
        
        # Search for the model name
        model = search_model_by_name(search_name)
        if model:
            return model
    
    return None

In [ ]:
inventory_id = ext_models[0]['inventory_id'] # enter desired inventory_id
print(inventory_id)

In [ ]:
# adjust the start_index start the model
start_index=0
# adjust the max_results to display the model
max_results = 20
# model name to search for
search_name = 'Desired External Model'

# Process all pages and get the found model
found_model = final_external_model_with_start_index_max_results(inventory_id, max_results, search_name)

# Output found model
print("Found model:", found_model)

In [ ]:
container_type = 'catalog' 
asset_id = found_model.get('asset_id')
container_id = found_model.get('inventory_id')  
external_model =facts_client.assets.get_model(model_id=asset_id, container_type=container_type, container_id=container_id)

---

## Deployment Methods

These deployment methods are specifically tailored for external models. They detail the processes and strategies for effectively deploying models into environments, ensuring they meet operational requirements and integrate smoothly with existing systems.

For more Details , refer to [Deployments Methods](https://s3.us.cloud-object-storage.appdomain.cloud/aifactsheets-client/doc_files/asset_model/Model/Deployments.html)


### Retrieve Deployments for External Models

This function allows you to access and view all existing deployments associated with the external models. It provides detailed information on each deployment, including their status and configuration.


In [23]:
deployments = external_model.get_deployments()
for deployment in deployments:
    print(deployment)

2024/08/09 21:08:51 INFO : Deployments retrieved successfully
{
  "id": "99ba2b29cf98ae401fa4443fd9618fbd",
  "name": "Loan Application Model Deployment",
  "type": "online",
  "scoring_endpoint": "https://runtime.sagemaker.us-east-2.amazonaws.com/endpoints/sagemaker-scikit-learn-2023-08-11-09-00-35-455/invocations",
  "is_deleted": false,
  "description": "Model deployed in AWS SageMaker to predict the loan application approval",
  "model_name": "Loan Application Model Test",
  "model_asset_id": "482d8c74-2655-4b17-aadb-eb45e1f56467",
  "model_container_type": "catalog",
  "model_container_id": "f8925e43-e661-4df7-8482-b6245433942c"
}


### Add New Deployment to External Model

This function allows you to add a deployment to the external model within the system. 

Note that this action will not affect external providers such as AWS or Azure. You can add multiple deployments simultaneously.




In [24]:
deployment_new = [{"id":"Loan Application Deploy","name":"Loan Application Model Deployment with modifications","type":"online","scoring_endpoint":endpoint_url,"description":"Model deployed in AWS SageMaker to predict the loan application approval"}]
deployments_list = external_model.add_deployments(deployment_new)

deployments_list[0]

2024/08/09 21:08:54 INFO : Deployment added successfully to an external model


{
  "id": "870f3666d8d6e8efdcede6fc53375cd3",
  "name": "Loan Application Model Deployment with modifications",
  "type": "online",
  "scoring_endpoint": "https://runtime.sagemaker.us-east-2.amazonaws.com/endpoints/sagemaker-scikit-learn-2023-08-11-09-00-35-455/invocations",
  "is_deleted": "false",
  "description": "Model deployed in AWS SageMaker to predict the loan application approval",
  "model_name": "Loan Application Model Test",
  "model_asset_id": "482d8c74-2655-4b17-aadb-eb45e1f56467",
  "model_container_type": "catalog",
  "model_container_id": "f8925e43-e661-4df7-8482-b6245433942c"
}

In [25]:
deployment_tuning = [{"id":"Loan Application Deploy With Tuning","name":"Loan Application Model Deployment with Tuning","type":"online","scoring_endpoint":endpoint_url,"description":"Model deployed in AWS SageMaker to predict the loan application approval"}]
deployments_list = external_model.add_deployments(deployment_tuning)

2024/08/09 21:08:55 INFO : Deployment added successfully to an external model


In [26]:
deployments_all = external_model.get_deployments()
for deployment in deployments_all:
    print(deployment)

2024/08/09 21:08:57 INFO : Deployments retrieved successfully
{
  "id": "99ba2b29cf98ae401fa4443fd9618fbd",
  "name": "Loan Application Model Deployment",
  "type": "online",
  "scoring_endpoint": "https://runtime.sagemaker.us-east-2.amazonaws.com/endpoints/sagemaker-scikit-learn-2023-08-11-09-00-35-455/invocations",
  "is_deleted": false,
  "description": "Model deployed in AWS SageMaker to predict the loan application approval",
  "model_name": "Loan Application Model Test",
  "model_asset_id": "482d8c74-2655-4b17-aadb-eb45e1f56467",
  "model_container_type": "catalog",
  "model_container_id": "f8925e43-e661-4df7-8482-b6245433942c"
}
{
  "id": "870f3666d8d6e8efdcede6fc53375cd3",
  "name": "Loan Application Model Deployment with modifications",
  "type": "online",
  "scoring_endpoint": "https://runtime.sagemaker.us-east-2.amazonaws.com/endpoints/sagemaker-scikit-learn-2023-08-11-09-00-35-455/invocations",
  "is_deleted": false,
  "description": "Model deployed in AWS SageMaker to pred

### Methods for  External Model Deployments

Explore the various methods for managing and configuring external model deployments.


In [27]:
print("Deployment name is : {}".format(deployment.get_name()))
print("Deployment ID is : {}".format(deployment.get_id()))
print("Deployment description is : {}".format(deployment.get_description()))
print("Deployment type is : {}".format(deployment.get_type()))
print("Deployment scoring endpoint is : {}".format(deployment.get_scoring_endpoint()))
print("Deployment model id is : {}".format(deployment.get_model_id()))
print("Deployment model name is : {}".format(deployment.get_model_name()))

Deployment name is : Loan Application Model Deployment with Tuning
Deployment ID is : 79fd918459f44f9eccb0b7712c8700a1
Deployment description is : Model deployed in AWS SageMaker to predict the loan application approval
Deployment type is : online
Deployment scoring endpoint is : https://runtime.sagemaker.us-east-2.amazonaws.com/endpoints/sagemaker-scikit-learn-2023-08-11-09-00-35-455/invocations
Deployment model id is : 482d8c74-2655-4b17-aadb-eb45e1f56467
Deployment model name is : Loan Application Model Test


### Update Deployment Attributes: Description, Scoring Endpoint, and Type


In [28]:
deployment.set_description(value="update - Model deployed in AWS SageMaker to predict the loan application approval")
deployment.set_scoring_endpoint(value="https://runtime.sagemaker.us-east-2.amazonaws.com/endpoints/sagemaker-scikit-learn-2023-08-11-09-00-35-455/invocations-update")
deployment.set_type("update-online")

2024/08/09 21:09:02 INFO : Description is updated for deployment successfully
2024/08/09 21:09:03 INFO : Scoring endpoint is updated for deployment successfully
2024/08/09 21:09:04 INFO : Type is updated for deployment under model_stub successfully
2024/08/09 21:09:04 INFO : Type is updated for deployment under modelfacts_system successfully


In [29]:
print("Deployment description is : {}".format(deployment.get_description()))
print("Deployment type is : {}".format(deployment.get_type()))
print("Deployment scoring endpoint is : {}".format(deployment.get_scoring_endpoint()))

Deployment description is : update - Model deployed in AWS SageMaker to predict the loan application approval
Deployment type is : update-online
Deployment scoring endpoint is : https://runtime.sagemaker.us-east-2.amazonaws.com/endpoints/sagemaker-scikit-learn-2023-08-11-09-00-35-455/invocations-update


### Delete Deployments

Users can delete multiple deployments simultaneously. 

This action performs a soft delete, updating the **`is_deleted`** attribute from `false` to `true`.


In [30]:
deployment_ids=[deployment.get_id()]
external_model.delete_deployments(deployment_ids)

2024/08/09 21:09:07 INFO : Deployment 79fd918459f44f9eccb0b7712c8700a1 deleted successfully


In [31]:
deployments_all = external_model.get_deployments()
for deployment in deployments_all:
    print(deployment)

2024/08/09 21:09:07 INFO : Deployments retrieved successfully
{
  "id": "99ba2b29cf98ae401fa4443fd9618fbd",
  "name": "Loan Application Model Deployment",
  "type": "online",
  "scoring_endpoint": "https://runtime.sagemaker.us-east-2.amazonaws.com/endpoints/sagemaker-scikit-learn-2023-08-11-09-00-35-455/invocations",
  "is_deleted": false,
  "description": "Model deployed in AWS SageMaker to predict the loan application approval",
  "model_name": "Loan Application Model Test",
  "model_asset_id": "482d8c74-2655-4b17-aadb-eb45e1f56467",
  "model_container_type": "catalog",
  "model_container_id": "f8925e43-e661-4df7-8482-b6245433942c"
}
{
  "id": "870f3666d8d6e8efdcede6fc53375cd3",
  "name": "Loan Application Model Deployment with modifications",
  "type": "online",
  "scoring_endpoint": "https://runtime.sagemaker.us-east-2.amazonaws.com/endpoints/sagemaker-scikit-learn-2023-08-11-09-00-35-455/invocations",
  "is_deleted": false,
  "description": "Model deployed in AWS SageMaker to pred



---
## Creation of New AI Use Case <a href="#add_mut"></a>

An **AI Use Case** tracks model asset lifecycles across environments like development, pre-production, and production. 

**Note:** The term "AI Use Case" has replaced "Model Use Case" to reflect a broader range of AI assets. While some APIs may still use the old terminology, it will be phased out.

For more information, refer to the [New AI Use Case](https://s3.us.cloud-object-storage.appdomain.cloud/aifactsheets-client/doc_files/asset_model/Model%20Usecase/Creation%20of%20Model%20Uscase.html).

- If `ai_usecase_id` is not provided, the default inventory_id is used (requires `EDITOR` access).

- Retrieve the AI Use Case ID from the URL in inventory or by using `get_ai_usecase()`.

- For Cloud Pak for Data, ensure OpenPages integration is disabled (create inventory permission needed).


In [32]:
ai_usecase_inventory_id = inventory.get_id()
ai_usecase_name="Automatic Loan classification"
ai_usecase_desc="AI usecase for Loan classification"

##### ⚠️ Warning: Use of `catalog_id`

- The `catalog_id` parameter can specify either an inventory ID or a catalog ID. Inventories, optimized for watsonx.governance, are recommended.


- Catalogs are still supported but will be deprecated over time. As a best practice, use inventories for storing use cases and external models.


In [33]:
ai_usecase = facts_client.assets.create_ai_usecase(catalog_id=ai_usecase_inventory_id,name=ai_usecase_name,description=ai_usecase_desc)
ai_usecase.get_info(True)

2024/08/09 21:09:24 INFO : AI usecase created successfully


{'name': 'Automatic Loan classification',
 'description': 'AI usecase for Loan classification',
 'asset_type': 'model_entry',
 'url': 'https://dataplatform.cloud.ibm.com/data/catalogs/f8925e43-e661-4df7-8482-b6245433942c/asset/4a4a7e80-3ad3-4f75-942b-fec55980c7b8?context=cpdaas',
 'model_usecase_id': '4a4a7e80-3ad3-4f75-942b-fec55980c7b8',
 'container_type': 'catalog',
 'catalog_id': 'f8925e43-e661-4df7-8482-b6245433942c',
 'facts_type': 'model_entry_user'}

---

## Create an Approach <a href="#createapproach"></a>

- Track multiple models and prompts under a single use case by grouping them into different approaches.
- Create multiple approaches for various classification algorithms to facilitate comparison and integration.
- Use approaches to manage different models that need to be combined for a specific use case.

For more information, refer to the [Approach Documentation](https://s3.us.cloud-object-storage.appdomain.cloud/aifactsheets-client/doc_files/asset_model/Model%20Usecase/Model%20Usecase%20Approach.html)


In [34]:
external_approach = ai_usecase.create_approach(name="externalapproach",description="approach for external AWS model")
external_approach.get_info()

2024/08/09 21:09:29 INFO : Approach created successfully


{'approach_id': '241c4708-ec99-4fe2-ba33-d6095d0bd1a8',
 'approach_name': 'externalapproach',
 'approach_desc': 'approach for external AWS model',
 'model_asset_id': '4a4a7e80-3ad3-4f75-942b-fec55980c7b8',
 'model_container_type': 'catalog',
 'model_container_id': 'f8925e43-e661-4df7-8482-b6245433942c'}

---
## Track a External Model Under an AI Use Case <a name="add_mu"></a>


- **AI Use Cases** are designed to monitor the lifecycle of model assets across various stages, including development, pre-production, and production.
- To effectively integrate a model into an AI use case, three critical elements must be addressed: the **model**, the **AI use case**, and the **approach**.
- Link an existing AI use case by using the following method: `model.track(usecase=<ai_usecase>, approach=<approach1>, version_number="<Version Number>")`.

- **Version Numbers** are categorized as follows:
    - **Major Version:** Indicates significant changes, represented as `1.0.0`.
    - **Minor Version:** Reflects incremental improvements, represented as `0.1.0`.
    - **Patch Version:** Denotes minor fixes or updates, represented as `0.0.1`.
    - **Custom Version:** Allows for tailored versioning according to specific user needs.

- Ensure that the `ai_usecase`, `approach`, and `version_number` parameters are provided as mandatory.

For more information, refer to the [Governing AI Assets](https://s3.us.cloud-object-storage.appdomain.cloud/aifactsheets-client/doc_files/AIGov/Tracking%20models%20with%20AI%20Factsheets.html)


In [35]:
external_model.track(usecase=ai_usecase,approach=external_approach,version_number="major",version_comment="external model major version")

-----------------------------------------------------------------------------------------------------------------------------
                                                   Tracking Process Started                                                  
-----------------------------------------------------------------------------------------------------------------------------
2024/08/09 21:09:33 INFO : Assigned externalapproach to Automatic Loan classification for tracking.
2024/08/09 21:09:33 INFO : Initiate linking model to existing AI usecase 4a4a7e80-3ad3-4f75-942b-fec55980c7b8
2024/08/09 21:09:40 INFO : Successfully finished linking Model 482d8c74-2655-4b17-aadb-eb45e1f56467 to AI usecase


{'model_entry_catalog_id': 'f8925e43-e661-4df7-8482-b6245433942c',
 'model_entry_id': '4a4a7e80-3ad3-4f75-942b-fec55980c7b8',
 'model_entry_name': 'Automatic Loan classification',
 'model_entry_status': 'Draft',
 'version_number': '1.0.0',
 'version_comment': 'external model major version',
 'approach_name': 'externalapproach'}

---

## External Model Environment Type <a name="existing_env_type"></a>

This section details the various environment types available for external models. It also discusses the process for promoting external model across different environment stages.

For more details , refer to [Managing the Lifecycle Phases of a Model](https://s3.us.cloud-object-storage.appdomain.cloud/aifactsheets-client/doc_files/AIGov/Model%20Env.html)


In [36]:
external_model.get_environment_type()

{'classification': 'TEST',
 'reason': 'The space type is development and deployment_details are available which is considered as TEST environment and asset shows under TEST stage'}

#### Model Promotion Across Pillars

Models move through different pillars based on their environment and evaluation status. Here’s how the transition works:

- **Develop**: The initial stage where models are created and developed.

- **Test**: Once promoted to a space, the model appears in the Test pillar.

- **Validate**: If a model's deployment is evaluated in Watson OpenScale and tagged as pre-production, it will be displayed in the Validate pillar. Additionally, models promoted to spaces with the `AIGovernance: Pre-production` tag will also be shown in the Validate pillar.

- **Operate**: Models tagged as production or those evaluated and tagged as production in Watson OpenScale will be displayed in the Operate pillar. Models promoted to spaces with the `AIGovernance: Production` tag will be shown in the Operate pillar.

#### -> Promote External Model from Test to Validate

In [37]:
external_model.set_environment_type(from_container="test",to_container="validate")

2024/08/09 21:10:04 INFO : Asset successfully moved from test to validate environment


#### ->Promote External Model from Validate to Operate

In [38]:
external_model.set_environment_type(from_container="validate",to_container="operate")

2024/08/09 21:10:15 INFO : Asset successfully moved from validate to operate environment


#### -> Promote External Model from Operate to Validate

In [39]:
external_model.set_environment_type(from_container="operate",to_container="validate")

2024/08/09 21:10:25 INFO : Asset successfully moved from operate to validate environment


---
## Deployment Environment Type <a name="5"></a>

This section outlines the scenarios for external model deployment environments and promotions:

- **Models Without Deployment**: These models are located in the Develop pillar, where they are in the initial development phase.
  
- **Models With Deployment**:
  - **`Multiple Deployments`**: Moving a single deployment results in the creation of a clone of the entire model in the target pillar, with the deployment placed there.
  
  - **`Single Deployment`**: Moving the deployment causes the entire model to be migrated to the target pillar.

- **Complete Model Migration**: When a model is moved to a target pillar, it includes all associated deployments, whether there is one or multiple. Once a model is in a target pillar, it cannot be moved further.

- **OpenScale Evaluations**: Deployments evaluated by OpenScale will remain in their current pillar and cannot be relocated to another pillar.

**Notes**:
- Models with deployments cannot be moved from the Test, Validate, or Operate Pillars back to the Develop Pillar.



For more details , refer to [Managing the Lifecycle Phases of a Model](https://s3.us.cloud-object-storage.appdomain.cloud/aifactsheets-client/doc_files/AIGov/Model%20Env.html)


In [40]:
deployments_all = external_model.get_deployments()
for deployment in deployments_all:
    print(deployment)

2024/08/09 21:11:00 INFO : Deployments retrieved successfully
{
  "id": "99ba2b29cf98ae401fa4443fd9618fbd",
  "name": "Loan Application Model Deployment",
  "type": "online",
  "scoring_endpoint": "https://runtime.sagemaker.us-east-2.amazonaws.com/endpoints/sagemaker-scikit-learn-2023-08-11-09-00-35-455/invocations",
  "is_deleted": false,
  "description": "Model deployed in AWS SageMaker to predict the loan application approval",
  "model_name": "Loan Application Model Test",
  "model_asset_id": "482d8c74-2655-4b17-aadb-eb45e1f56467",
  "model_container_type": "catalog",
  "model_container_id": "f8925e43-e661-4df7-8482-b6245433942c"
}
{
  "id": "870f3666d8d6e8efdcede6fc53375cd3",
  "name": "Loan Application Model Deployment with modifications",
  "type": "online",
  "scoring_endpoint": "https://runtime.sagemaker.us-east-2.amazonaws.com/endpoints/sagemaker-scikit-learn-2023-08-11-09-00-35-455/invocations",
  "is_deleted": false,
  "description": "Model deployed in AWS SageMaker to pred

In [41]:
deployments_all[1].set_environment_type(to_container="operate")

2024/08/09 21:11:18 INFO : Deployment successfully moved to operate environment


---
## Cleanup <a href="#clean"></a>

In [42]:
if run_cleanup_at_end:
    # remove model and model usecase
    external_model.untrack()
    facts_client.assets.remove_asset(asset_id=ai_usecase.get_info()["model_usecase_id"],container_type=ai_usecase.get_info()["container_type"],container_id=ai_usecase.get_info()["catalog_id"])
    facts_client.assets.remove_asset(asset_id=external_model.get_info()["asset_id"],container_type=external_model.get_info()["container_type"],container_id=external_model.get_info()["container_id"])

else:
    
    model_ui_url = external_model.get_info(verbose=True)["url"]
    display(Markdown("[Click here to see the created wml model details in UI](" + model_ui_url + ")"))
    model_usecase_ui_url = ai_usecase.get_info(verbose=True)["url"]
    display(Markdown("[Click here to see the created model use case in UI](" + model_usecase_ui_url + ")"))

2024/08/09 21:11:56 INFO : Successfully finished unregistering WKC Model 482d8c74-2655-4b17-aadb-eb45e1f56467 from model use case.
2024/08/09 21:11:57 INFO : Successfully deleted asset id 4a4a7e80-3ad3-4f75-942b-fec55980c7b8 in catalog f8925e43-e661-4df7-8482-b6245433942c
2024/08/09 21:11:58 INFO : Successfully deleted asset id 482d8c74-2655-4b17-aadb-eb45e1f56467 in catalog f8925e43-e661-4df7-8482-b6245433942c



### Delete Inventory

This section covers the process for deleting an inventory item. Make sure to carefully review the item before proceeding, as deletion is irreversible.


In [ ]:
inventory.delete_inventory()

**Created by:**  


IBM watsonx.governance - *AI Factsheet Python SDK Team*

---

**Copyright © 2022-2024 IBM**  
Released under the [Apache 2.0 License](https://www.apache.org/licenses/LICENSE-2.0).
